In [1]:
from __future__ import division
import glob, os, gc
import os.path
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn.decomposition import (PCA, RandomizedPCA)
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from IPython.display import Image
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import pybedtools
import pysam
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
Path_SLAV = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/SLAV_Data/" 
Path_Common = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/"
narrowpeak = "-ready_peaks.narrowPeak"
peaks_merged = "_peaksMerged.txt"
peaks_merged_bed = "_peaksMerged.bed"
peaks_correct_bed = "_peaksCorrect.bed"
peakregions_sml = ".peakregions_sml"
peakregions_lrg = ".peakregions_lrg"
peaks_correct_data = "_peaksCorrect.data"
peaks_L1HS_bedgraph = "_peaks_L1HS_mapped.bedgraph"
loci_sml = ".loci_sml"
loci_lrg = ".loci_lrg"
overlap = "_overlap_"
overlap_sml = "_overlap_sml_"
overlap_lrg = "_overlap_lrg_"
bam = "-ready.bam"
bed = ".bed"
## rmask Paths 
L1HS = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1HS_Final.bed"
L1PA2 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1PA2.bed"
L1PA3 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1PA3.bed"
L1PA4 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1PA4.bed"
L1PA5 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1PA5.bed"
L1PA2345 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1PA2345_Final.bed"
L1_Other = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1_Other_Final.bed"

##SLAV Data
Bulk_1571_Cerebellum = "1571_cereb_BT_40_L3"
Bulk_1571_Hippocampus = "1571_hippo_BT_41_L3"
Bulk_1846_Cerebellum = "1846_cereb_BT_13_L3"
Bulk_1846_Cortex = "1846_cortex_BT_71_L3"
Bulk_1846_Hippocampus = "1846_hippo_BT_19_L3"
Bulk_1846_Liver = "1846_liver_BT_22_L3"
Bulk_5125_Cortex = "5125_cortex_BT_122_L3"
Bulk_5125_Hippocampus = "5125_hippo_BT_139_L3"
Bulk_5125_Liver = "5125_liver_BT_164_L3"
SC_1571_Hippo = ["1571_hippo_SC_43_L3","1571_hippo_SC_45_L3","1571_hippo_SC_46_L3","1571_hippo_SC_47_L3","1571_hippo_SC_48_L3","1571_hippo_SC_50_L3","1571_hippo_SC_51_L3","1571_hippo_SC_52_L3","1571_hippo_SC_53_L3","1571_hippo_SC_55_L3","1571_hippo_SC_56_L3","1571_hippo_SC_57_L3","1571_hippo_SC_58_L3","1571_hippo_SC_59_L3","1571_hippo_SC_61_L3","1571_hippo_SC_62_L3","1571_hippo_SC_63_L3","1571_hippo_SC_64_L3"]
SC_1846_Cortex = ["1846_cortex_SC_72_L3","1846_cortex_SC_73_L3","1846_cortex_SC_74_L3","1846_cortex_SC_75_L3","1846_cortex_SC_78_L3","1846_cortex_SC_79_L3","1846_cortex_SC_80_L3","1846_cortex_SC_81_L3","1846_cortex_SC_82_L3","1846_cortex_SC_83_L3","1846_cortex_SC_84_L3","1846_cortex_SC_85_L3","1846_cortex_SC_86_L3"]
SC_1846_Hippo = ["1846_hippo_SC_100_L3","1846_hippo_SC_101_L3","1846_hippo_SC_102_L3","1846_hippo_SC_103_L3","1846_hippo_SC_104_L3","1846_hippo_SC_105_L3","1846_hippo_SC_106_L3","1846_hippo_SC_107_L3","1846_hippo_SC_108_L3","1846_hippo_SC_109_L3","1846_hippo_SC_110_L3","1846_hippo_SC_111_L3","1846_hippo_SC_112_L3","1846_hippo_SC_113_L3","1846_hippo_SC_88_L3","1846_hippo_SC_89_L3","1846_hippo_SC_90_L3","1846_hippo_SC_91_L3","1846_hippo_SC_92_L3","1846_hippo_SC_93_L3","1846_hippo_SC_94_L3","1846_hippo_SC_95_L3","1846_hippo_SC_99_L3"]
SC_5125_Cortex = ["5125_cortex_SC_125_L3","5125_cortex_SC_126_L3","5125_cortex_SC_127_L3","5125_cortex_SC_128_L3","5125_cortex_SC_129_L3","5125_cortex_SC_130_L3","5125_cortex_SC_131_L3","5125_cortex_SC_132_L3","5125_cortex_SC_133_L3","5125_cortex_SC_134_L3","5125_cortex_SC_135_L3","5125_cortex_SC_136_L3","5125_cortex_SC_138_L3"]
SC_5125_Hippo = ["5125_hippo_SC_140_L3","5125_hippo_SC_141_L3","5125_hippo_SC_142_L3","5125_hippo_SC_143_L3","5125_hippo_SC_144_L3","5125_hippo_SC_145_L3","5125_hippo_SC_147_L3","5125_hippo_SC_149_L3","5125_hippo_SC_150_L3","5125_hippo_SC_151_L3","5125_hippo_SC_152_L3","5125_hippo_SC_153_L3","5125_hippo_SC_154_L3","5125_hippo_SC_155_L3","5125_hippo_SC_156_L3","5125_hippo_SC_157_L3","5125_hippo_SC_158_L3","5125_hippo_SC_159_L3","5125_hippo_SC_160_L3","5125_hippo_SC_161_L3","5125_hippo_SC_162_L3","5125_hippo_SC_163_L3"]
Data_Sets_SLAV = []
Data_Sets_SLAV.append([SC_1571_Hippo,Bulk_1571_Hippocampus,Bulk_1571_Cerebellum])
Data_Sets_SLAV.append([SC_1846_Cortex,Bulk_1846_Cortex,Bulk_1846_Liver])
Data_Sets_SLAV.append([SC_1846_Hippo,Bulk_1846_Hippocampus,Bulk_1846_Liver])
Data_Sets_SLAV.append([SC_5125_Cortex,Bulk_5125_Cortex,Bulk_5125_Liver])
Data_Sets_SLAV.append([SC_5125_Hippo,Bulk_5125_Hippocampus,Bulk_5125_Liver])

##Common Data
Bulk_Brain_Common = "04132016_mw_Bulk_cor"
Bulk_Fibro_Common = "05252016_mw_Bulk_fib"
SC_MDA = ["04132016_mw_1571_SC_B4_S48","04132016_mw_1571_SC_D4_S49","04132016_mw_L1B1_SC_A2_S43","04132016_mw_L1B1_SC_C1_S45","04132016_mw_L1B1_SC_C2_S46","04132016_mw_L1B1_SC_D2_S50","04132016_mw_L1B1_SC_E2_S51","04132016_mw_L1B1_SC_E3_S52","04132016_mw_L1B1_SC_E3_S52","04132016_mw_L1B1_SC_F2_S53","04132016_mw_L1B1_SC_G1_S54","04132016_mw_L1B1_SC_H1_S55","05252016_mw_L1B1_SC_B4_S47"] 
SC_MALBAC = ["2122_S1","2178_S2","2179_S3","2180_S4","2184_S5","2186_S6","2187_S7","2188_S8","2193_S9","2196_S10","2197_S11","2198_S12","2261_S13","2263_S14","2264_S15","2265_S16"] 
Data_Sets_Common = []
Data_Sets_Common.append([SC_MALBAC,Bulk_Brain_Common,Bulk_Fibro_Common])
Data_Sets_Common.append([SC_MDA,Bulk_Brain_Common,Bulk_Fibro_Common])

In [ ]:
for dset in Data_Sets_SLAV:  
    Bulk_Brain_Peaks = os.path.join(Path_SLAV,  dset[1], "peaks.bed")
    Bulk_NonBrain_Peaks = os.path.join(Path_SLAV,  dset[2], "peaks.bed")
    a = pybedtools.BedTool(Bulk_Brain_Peaks)
    b_L1HS = pybedtools.BedTool(L1HS)
    L1HS_in_Bulk = b_L1HS.intersect(a)
    output = "Percent L1HS Overlap: " + str((len(L1HS_in_Bulk) / len(b_L1HS))*100)
    print output
    b_L1PA2 = pybedtools.BedTool(L1PA2)
    L1PA2_in_Bulk = b_L1PA2.intersect(a)
    output = "Percent L1PA2 Overlap: " + str((len(L1PA2_in_Bulk) / len(b_L1PA2))*100)
    print output
    b_L1PA3 = pybedtools.BedTool(L1PA3)
    L1PA3_in_Bulk = b_L1PA3.intersect(a)
    b_L1PA4 = pybedtools.BedTool(L1PA4)
    L1PA4_in_Bulk = b_L1PA4.intersect(a)
    b_L1PA5 = pybedtools.BedTool(L1PA5)
    L1PA5_in_Bulk = b_L1PA5.intersect(a)      
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(Path_SLAV, cell))
        name = os.path.join(Path_SLAV, cell, cell)
        bamfile = os.path.join(Path_SLAV, cell, cell+bam)
        Cell_Peaks = os.path.join(Path_SLAV, cell, "peaks.bed")
        a = pybedtools.BedTool(Cell_Peaks)
        L1HS_in_Cell = L1HS_in_Bulk.intersect(a)
        output = "Percentage of Bulk L1HS detected in Cell: " + str((len(L1HS_in_Cell) / len(L1HS_in_Bulk))*100)
        print output
        L1PA2_in_Cell = L1PA2_in_Bulk.intersect(a)
        output = "Percentage of Bulk L1PA2 detected in Cell: " + str((len(L1PA2_in_Cell) / len(L1PA2_in_Bulk))*100)
        print output
        L1HS_in_Cell_slop1K = L1HS_in_Cell.slop(b=1000, genome='hg19')
        a_bam = pysam.AlignmentFile(os.path.join(Path_SLAV, cell, cell + bam), 'rb')
        bam_reads=0
        for read in a_bam.fetch(): bam_reads+=1
        output = "Total Aligned Reads: " + str(bam_reads)  
        print output
        L1HS_reads=0
        for interval in L1HS_in_Cell_slop1K:
            for read in a_bam.fetch(str(interval[0]), int(interval[1]), int(interval[2])): L1HS_reads+=1 
        output = "average aligned reads per L1HS: " + str((L1HS_reads / len(L1HS_in_Cell)))
        print output
        #output = "Percent aligned reads that map to L1HS: " + str((L1HS_reads / bam_reads)*100)
        #print output
        #output = "average L1HS reads as percent of total: " + str(((L1HS_reads / len(L1HS_in_Cell)) / bam_reads)*100)
        #print output
        L1PA2_in_Cell_slop1K = L1PA2_in_Cell.slop(b=1000, genome='hg19')
        L1PA2_reads=0
        for interval in L1PA2_in_Cell_slop1K:
            for read in a_bam.fetch(str(interval[0]), int(interval[1]), int(interval[2])): L1PA2_reads+=1 
        output = "average aligned reads per L1PA2: " + str((L1PA2_reads / len(L1PA2_in_Cell)))
        print output
        #output = "Percent aligned reads that map to L1PA2: " + str((L1PA2_reads / bam_reads)*100)
        #print output
        #output = "average L1PA2 reads as percent of total: " + str(((L1PA2_reads / len(L1PA2_in_Cell)) / bam_reads)*100)
        #print output
        
        #percentage of reads on-target

Percent L1HS Overlap: 60.9455958549
Percent L1PA2 Overlap: 66.6870042709
1571_hippo_SC_43_L3
Percentage of Bulk L1HS detected in Cell: 94.3676939426
Percentage of Bulk L1PA2 detected in Cell: 89.4784995425
Total Aligned Reads: 8862220
average aligned reads per L1HS: 1393.31531532
average aligned reads per L1PA2: 871.66325835
1571_hippo_SC_45_L3
Percentage of Bulk L1HS detected in Cell: 92.6673751328
Percentage of Bulk L1PA2 detected in Cell: 87.0692284233
Total Aligned Reads: 12100577
average aligned reads per L1HS: 1573.48394495
average aligned reads per L1PA2: 1143.97723292
1571_hippo_SC_46_L3
Percentage of Bulk L1HS detected in Cell: 97.5557917109
Percentage of Bulk L1PA2 detected in Cell: 80.6038426349
Total Aligned Reads: 3382735
average aligned reads per L1HS: 1933.49237473
average aligned reads per L1PA2: 161.220204313
1571_hippo_SC_47_L3
Percentage of Bulk L1HS detected in Cell: 89.4792773645
Percentage of Bulk L1PA2 detected in Cell: 83.043610857
Total Aligned Reads: 9393969
a